In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [48]:
reviews = pd.read_csv("reviews.csv", delimiter=",", encoding="latin1", nrows=2000)
recipes = pd.read_csv("recipes.csv", delimiter=",", encoding="latin1", nrows=10000)

In [49]:
reviews

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
...,...,...,...,...,...,...,...,...
1995,4035,12116,20236,Scott Bakel,4,"The sauce was the best, I will try it on alot ...",2001-10-01T11:08:49Z,2001-10-01T11:08:49Z
1996,4036,11125,16856,Vickey Wickenhauser,5,My family and I have been been making horserad...,2001-10-01T11:08:49Z,2001-10-01T11:08:49Z
1997,4037,8709,7308,Kat2355,0,If you're just making one or two (or if you do...,2001-10-01T11:08:49Z,2001-10-01T11:08:49Z
1998,4038,2783,7308,Kat2355,5,"I used light cream cheese, and light sour crea...",2001-10-01T11:08:49Z,2001-10-01T11:08:49Z


Création de la matrice Utilisateur-Recette

In [50]:
def create_user_item(dataframe):
    user_item = dataframe.pivot(index='AuthorId', columns='RecipeId', values='Rating')
    return user_item


In [51]:
user_item_matrix = create_user_item(reviews)
user_item_matrix = user_item_matrix.fillna(0)
user_item_matrix.sum()

RecipeId
44       19.0
67        5.0
83        6.0
100       6.0
104       3.0
         ... 
51964     9.0
52727     5.0
54484     1.0
64306     4.0
70137    10.0
Length: 1329, dtype: float64

Calcul de la similarité entre utilisateurs

In [52]:
user_item_matrix 

RecipeId,44,67,83,100,104,113,129,131,142,144,...,24533,29062,37326,42917,48296,51964,52727,54484,64306,70137
AuthorId,,,,,,,,,,,,,,,,,,,,,
1566,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
# Calcul de la similarité de cosinus entre les utilisateurs
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)


In [54]:
user_similarity

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [55]:
user_item_matrix.sum()

RecipeId
44       19.0
67        5.0
83        6.0
100       6.0
104       3.0
         ... 
51964     9.0
52727     5.0
54484     1.0
64306     4.0
70137    10.0
Length: 1329, dtype: float64

In [56]:
import numpy as np

# Fonction de recommandation basée sur les utilisateurs avec ajout des retours de recommandations
def recommend_recipes_user_based(user_id, user_item_matrix, user_similarity, top_n=5):
    user_id_index = user_item_matrix.index.get_loc(user_id)
    recommendations = []  # Liste pour stocker les recommandations
    
    for recipe_id in user_item_matrix.columns:
        if user_item_matrix.loc[user_id, recipe_id] == 0:  # Si l'utilisateur n'a pas encore évalué cette recette
            prediction = 0
            sum_similarities = 0
            for authorID in user_item_matrix.index:
                if authorID != user_id:
                    # On filtre les notes supérieures à 3 et les similarités supérieures à 0.2 ou inférieures à -0.2
                    if user_item_matrix.loc[authorID, recipe_id] > 3:
                        similarity = user_similarity[user_id_index][user_item_matrix.index.get_loc(authorID)]
                        if similarity > 0.2 or similarity <= -0.2:
                            prediction += user_item_matrix.loc[authorID, recipe_id] * similarity
                            sum_similarities += similarity

            if sum_similarities > 0:  # Éviter une division par zéro
                prediction /= sum_similarities
                recommendations.append((recipe_id, prediction))  # Ajouter la recette et la prédiction
                
    # Trier les recommandations par score de prédiction et garder les top_n
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)[:top_n]
    
    # Affichage des prédictions
    for recipe_id, score in recommendations:
        print(f"Prediction for recipe {recipe_id} is: {score}")
    
    return recommendations  # Retourner les top_n recommandations


# Exemple d'utilisation avec un utilisateur aléatoire
random_user_id = np.random.choice(user_item_matrix.index)
recommendations = recommend_recipes_user_based(random_user_id, user_item_matrix, user_similarity, top_n=5)

# Affichage des recommandations
for recipe_id, score in recommendations:
    print(f"Recommended Recipe ID: {recipe_id} with prediction score: {score}")



In [57]:
# Fonction pour évaluer les recommandations basées sur les utilisateurs
def evaluate_user_based(user_similarity_matrix, user_item_matrix, test_data, top_n=5):
    actual_ratings = []
    predicted_ratings = []

    # Parcourir les utilisateurs du jeu de test
    for user_id in test_data['AuthorId'].unique():
        if user_id in user_item_matrix.index:
            # Obtenir les recettes et notes pour cet utilisateur dans le jeu de test
            user_test_data = test_data[test_data['AuthorId'] == user_id]
            recipes_tested = user_test_data['RecipeId'].values
            actual_ratings.extend(user_test_data['Rating'].values)
            
            # Recommandations pour cet utilisateur
            recommendations = recommend_recipes_user_based(user_id, user_item_matrix, user_similarity)
            
            # Trouver les recettes recommandées dans le jeu de test
            recommended_recipe_ids = [recipe_id for recipe_id, score in recommendations]
            
            # Calculer les scores des prédictions pour les recettes dans le jeu de test
            user_pred_ratings = [
                user_item_matrix.loc[user_id, recipe] if recipe in recommended_recipe_ids else 0
                for recipe in recipes_tested
            ]
            predicted_ratings.extend(user_pred_ratings)

    # Calculer les métriques d'évaluation
    rmse = mean_squared_error(actual_ratings, predicted_ratings, squared=False)
    mae = mean_absolute_error(actual_ratings, predicted_ratings)
    
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    
    return rmse, mae

# Exemple d'utilisation avec un sous-ensemble des données de test
test_data = reviews.sample(frac=0.1, random_state=42)  # 10% des données pour le test
rmse, mae = evaluate_user_based(user_similarity, user_item_matrix, test_data, top_n=5)

Prediction for recipe 10219 is: 5.0
Prediction for recipe 10198 is: 4.0
Prediction for recipe 2886 is: 5.000000000000001
Prediction for recipe 10626 is: 5.0
Prediction for recipe 155 is: 5.0
Prediction for recipe 8438 is: 5.0
Prediction for recipe 10084 is: 5.0
Prediction for recipe 10422 is: 5.0
Prediction for recipe 7682 is: 4.0
Prediction for recipe 12134 is: 5.0
Prediction for recipe 3866 is: 5.0
Prediction for recipe 4829 is: 5.0
Prediction for recipe 330 is: 4.0


KeyboardInterrupt: 